In [ ]:
%cd /content
!git clone https://github.com/aimacode/aima-python.git 
%cd /content/aima-python/
!git clone https://github.com/aimacode/aima-data.git
!pip install ipythonblocks qpsolvers

In [1]:
from agents import *
from notebook import psource
from collections import Counter

# CONTENTS

1. Overview
   >1.1 Agent   
   >1.2 Environment   
      
2. Agents in a 2-D Environment
   > 2.1 BlindDog in Graph2D Environment    
   > 2.2 EnergeticBlindDog in Graph2D Environment
3. Vacuum Cleaner Environment
   > 3.0 TrivialVacuumEnvironment   
   > 3.1 Random Agent Program   
   > 3.2 Table-Driven Agent Program   
   > 3.3 Simple Reflex Agent Program   
   > 3.4 Model-Based Reflex Agent Program   

# 1. Overview

## 1.1 AGENT

In [2]:
psource(Agent)

## 1.2 ENVIRONMENT

In [3]:
psource(Environment)

# 2. AGENTS IN A 2D ENVIRONMENT

In [2]:
class GraphicEnvironment(XYEnvironment):
    def __init__(self, width=10, height=10, boundary=True, color={}, display=False):
        """Define all the usual XYEnvironment characteristics,
        but initialise a BlockGrid for GUI too."""

        super().__init__(width, height)
        self.grid = BlockGrid(width+1, height+1, fill=(200, 200, 200))
        if display:
            self.grid.show()
            self.visible = True
        else:
            self.visible = False
        self.bounded = boundary
        self.colors = color

    def get_world(self):
        """Returns all the items in the world in a format
        understandable by the ipythonblocks BlockGrid."""
        result = []
        x_start, y_start = (0, 0)
        x_end, y_end = self.width+1, self.height+1
        for x in range(x_start, x_end):
            row = []
            for y in range(y_start, y_end):
                row.append(self.list_things_at((x, y)))
            result.append(row)
        return result

    """
    def run(self, steps=1000, delay=1):
        "" "Run the Environment for given number of time steps,
        but update the GUI too." ""
        for step in range(steps):
            sleep(delay)
            if self.visible:
                self.reveal()
            if self.is_done():
                if self.visible:
                    self.reveal()
                return
            self.step()
        if self.visible:
            self.reveal()
    """

    def run(self, steps=1000, delay=1):
        """Run the Environment for given number of time steps,
        but update the GUI too."""
        for step in range(steps):
            self.update(delay)
            if self.is_done():
                break
            self.step()
        self.update(delay)

    def update(self, delay=1):
        sleep(delay)
        self.reveal()

    def reveal(self):
        """Display the BlockGrid for this world - the last thing to be added
        at a location defines the location color."""
        self.draw_world()
        # wait for the world to update and
        # apply changes to the same grid instead
        # of making a new one.
        clear_output(1)
        self.grid.show()
        self.visible = True

    def draw_world(self):
        self.grid[:] = (200, 200, 200)
        world = self.get_world()
        for x in range(0, len(world)):
            for y in range(0, len(world[x])):
                if len(world[x][y]):
                    self.grid[y, x] = self.colors[world[x][y][-1].__class__.__name__]

    def conceal(self):
        """Hide the BlockGrid for this world"""
        self.visible = False
        display(HTML(''))

In [3]:
class Food(Thing):
    pass

class Water(Thing):
    pass

class Park2D(GraphicEnvironment):
    def percept(self, agent):
        things = self.list_things_at(agent.location)
        return things
    
    def execute_action(self, agent, action):
        if action == "move down":
            print('{} decided to {} at location: {}'.format(str(agent)[1:-1], action, agent.location))
            agent.movedown()
        elif action == "eat":
            items = self.list_things_at(agent.location, tclass=Food)
            if len(items) != 0:
                if agent.eat(items[0]): #Have the dog eat the first item
                    print('{} ate {} at location: {}'
                          .format(str(agent)[1:-1], str(items[0])[1:-1], agent.location))
                    self.delete_thing(items[0]) #Delete it from the Park after.
        elif action == "drink":
            items = self.list_things_at(agent.location, tclass=Water)
            if len(items) != 0:
                if agent.drink(items[0]): #Have the dog drink the first item
                    print('{} drank {} at location: {}'
                          .format(str(agent)[1:-1], str(items[0])[1:-1], agent.location))
                    self.delete_thing(items[0]) #Delete it from the Park after.
                    
    def is_done(self):
        no_edibles = not any(isinstance(thing, Food) or isinstance(thing, Water) for thing in self.things)
        dead_agents = not any(agent.is_alive() for agent in self.agents)
        return dead_agents or no_edibles

In [4]:
class BlindDog(Agent):
    location = [0,1] # change location to a 2d value
    direction = Direction("down") # variable to store the direction our dog is facing # 방향 정해주는 것
    
    def movedown(self):
        self.location[1] += 1 # move 다운으로 하면 location을 변화시켜줌
        
    def eat(self, thing):
        '''returns True upon success or False otherwise'''
        if isinstance(thing, Food):
            return True
        return False
    
    def drink(self, thing):
        ''' returns True upon success or False otherwise'''
        if isinstance(thing, Water):
            return True
        return False

In [5]:
def program(percepts):
    for p in percepts:
        if isinstance(p, Food):
            return 'eat'
        elif isinstance(p, Water):
            return 'drink'
    return 'move down'

### 2.1 BlindDog in Graph2D Environment

### >park 환경 선언 (너비: 2 / 길이:20 / BlindDog의 색: 빨간색 / Water의 색: 파란색 / Food의 색: 주황색

In [6]:
park = Park2D(width = 1, height = 19, display = True, color={'BlindDog': (200,0,0), 'Water': (0, 200, 200), 'Food': (230, 115, 40)})

,
,
,
,
,
,
,
,
,
,
,


### >Park에 agent, food, water 넣어줌 / 20 step 실시

In [7]:
dog = BlindDog(program)  
dogfood = Food()  
water = Water()  
park.add_thing(dog, [0,1]) 
park.add_thing(dogfood, [0,5]) 
park.add_thing(water, [0,7]) 

morewater = Water()
park.add_thing(morewater, [0,15]) 

steps = 20
park.run(steps)

,
,
,
,
,
,
,
,
,
,
,


### 2.2 EnergeticBlindDog in Graph2D Environment

## PROGRAM - EnergeticBlindDog

<table>
    <tr>
        <td><b>Percept:</b> </td>
        <td>Feel Food </td>
        <td>Feel Water</td>
        <td>Feel Nothing</td>
   </tr>
   <tr>
       <td><b>Action:</b> </td>
       <td>eat</td>
       <td>drink</td>
       <td>
       <table>
           <tr>
               <td><b>Remember being at Edge : </b></td>
               <td>At Edge</td>
               <td>Not at Edge</td>
           </tr>
           <tr>
               <td><b>Action : </b></td>
               <td>Turn Left / Turn Right <br> ( 50% - 50% chance )</td>
               <td>Turn Left / Turn Right / Move Forward <br> ( 25% - 25% - 50% chance )</td>
           </tr>
       </table>
       </td>
   </tr>
        
</table>

### > 단방향이 아닌 여러 방향으로 움직이는 BlindDog

In [8]:
class Park2D(GraphicEnvironment):
    def percept(self, agent):
        '''return a list of things that are in our agent's location'''
        things = self.list_things_at(agent.location)
        loc = copy.deepcopy(agent.location) # find out the target location
        #Check if agent is about to bump into a wall
        if agent.direction.direction == Direction.R:
            loc[0] += 1
        elif agent.direction.direction == Direction.L:
            loc[0] -= 1
        elif agent.direction.direction == Direction.D:
            loc[1] += 1
        elif agent.direction.direction == Direction.U:
            loc[1] -= 1
            
        if not self.is_inbounds(loc):
            things.append(Bump())        
        return things

    def execute_action(self, agent, action):
        '''에이전트가 수행하는 작업에 따라 환경 상태를 변경'''
                
        if action == 'turnright':
            print('{} decided to {} at location: {}'.format(str(agent)[1:-1], action, agent.location))
            agent.turn(Direction.R)
            
        elif action == 'turnleft':
            print('{} decided to {} at location: {}'.format(str(agent)[1:-1], action, agent.location))
            agent.turn(Direction.L)
        
        elif action == 'moveforward':
            print(agent.direction.direction)
            print('{} decided to move {}wards at location: {}'.format(str(agent)[1:-1], agent.direction.direction, agent.location))
            agent.moveforward()
        
        elif action == "eat": 
            items = self.list_things_at(agent.location, tclass=Food)
            if len(items) != 0:
                if agent.eat(items[0]):
                    print('{} ate {} at location: {}'
                          .format(str(agent)[1:-1], str(items[0])[1:-1], agent.location))
                    self.delete_thing(items[0])
        
        elif action == "drink": 
            items = self.list_things_at(agent.location, tclass=Water)
            if len(items) != 0:
                if agent.drink(items[0]):
                    print(f'{str(agent)[1:-1]} drank {str(items[0])[1:-1]} at location: {agent.location}')
                    self.delete_thing(items[0])
        
    def is_done(self): #종료 조건
        '''By default, we're done when we can't find a live agent, 
        but to prevent killing our cute dog, we will stop before itself - when there is no more food or water'''
        no_edibles = not any(isinstance(thing, Food) or isinstance(thing, Water) for thing in self.things)
        dead_agents = not any(agent.is_alive() for agent in self.agents)
        return dead_agents or no_edibles

In [9]:
from random import choice

class EnergeticBlindDog(Agent):
    location = [0,1]
    direction = Direction("down") 
    
    def moveforward(self, success=True):
        '''moveforward possible only if success (i.e. valid destination location)'''
        if not success:
            return
        if self.direction.direction == Direction.R:
            self.location[0] += 1
        elif self.direction.direction == Direction.L:
            self.location[0] -= 1
        elif self.direction.direction == Direction.D:
            self.location[1] += 1
        elif self.direction.direction == Direction.U:
            self.location[1] -= 1
    
    def turn(self, d):
        self.direction = self.direction + d
        
    def eat(self, thing):
        '''returns True upon success or False otherwise'''
        if isinstance(thing, Food):
            return True
        return False
    
    def drink(self, thing):
        ''' returns True upon success or False otherwise'''
        if isinstance(thing, Water):
            return True
        return False

In [10]:
def program(percepts):
    '''Returns an action based on it's percepts'''       
    for p in percepts: # first eat or drink - you're a dog!
        if isinstance(p, Food):
            return 'eat'
        elif isinstance(p, Water):
            return 'drink'

        if isinstance(p,Bump): # then check if you are at an edge and have to turn
            turn = False
            choice = random.choice((1,2));  # 1-right, 2-left, others-forward
        else:
            choice = random.choice((1,2,3,4)) # 1-right, 2-left, others-forward

    # choice 1이면 오른쪽으로 2이면 왼쪽으로 아니면 기본 방향 대로

    if choice == 1:
        return 'turnright'
    
    elif choice == 2:
        return 'turnleft'
    
    else:
        return 'moveforward'

### ENVIRONMENT - Park2D

### >park 환경 선언 (너비: 8 / 길이:8 / BlindDog의 색: 빨간색 / Water의 색: 주황색 / Food의 색: 파란색

In [11]:
park = Park2D(7, 7, display = True, color={'EnergeticBlindDog': (200,0,0), 'Water': (0, 200, 200), 'Food': (230, 115, 40)})

,,,,,,,
,,,,,,,
,,,,,,,
,,,,,,,
,,,,,,,
,,,,,,,
,,,,,,,
,,,,,,,


### >Park에 들어갈 EnergeticBlindDog, Food, Water 각각의 객체를 선언 / 20 step 실시

In [12]:
dog = EnergeticBlindDog(program)
dogfood = Food()
water = Water()

park.add_thing(dog, [0,0])
park.add_thing(dogfood, [1,2])
park.add_thing(water, [0,1])

morewater = Water()
morefood = Food()
park.add_thing(morewater, [2,4])
park.add_thing(morefood, [4,3])
park.run(20)

,,,,,,,
,,,,,,,
,,,,,,,
,,,,,,,
,,,,,,,
,,,,,,,
,,,,,,,
,,,,,,,


# 3. THE VACUUM WORLD   

## 3.0 Trivial Vacuum  Environment

In [13]:
class TrivialVacuumEnvironment(Environment):
    """This environment has two locations, A and B. Each can be Dirty
    or Clean. The agent perceives its location and the location's
    status. This serves as an example of how to implement a simple
    Environment."""

    def __init__(self):
        super().__init__()
        self.status = {loc_A: random.choice(['Clean', 'Dirty']),
                       loc_B: random.choice(['Clean', 'Dirty'])}
        self.results = []

    def thing_classes(self):
        return [Wall, Dirt, ReflexVacuumAgent, RandomVacuumAgent, TableDrivenVacuumAgent, ModelBasedVacuumAgent]

    def percept(self, agent):
        """Returns the agent's location, and the location status (Dirty/Clean)."""
        return agent.location, self.status[agent.location]

    def execute_action(self, agent, action):
        """Change agent's location and/or location's status; track performance.
        Score 10 for each dirt cleaned; -1 for each move."""
        if action == 'Right':
            agent.location = loc_B
            agent.performance -= 1
        elif action == 'Left':
            agent.location = loc_A
            agent.performance -= 1
        elif action == 'Suck':
            if self.status[agent.location] == 'Dirty':
                agent.performance += 10
            self.status[agent.location] = 'Clean'
        print(' 에이전트의 위치:', agent.location,'\n','에이전트의 행동:',action, '\n','환경의 퍼포먼스:',agent.performance,'\n', 
             '환경의 상태:', self.status)
        print('----------------------------------------------')
        self.results.append(action)
    def default_location(self, thing):
        """Agents start in either location at random."""
        return random.choice([loc_A, loc_B])
    
    #def is_done(self): #종료 조건
    #    done = self.status == {(0, 0): 'Clean', (1, 0): 'Clean'}
    #    return done

## 3.1 Random Agent Program

In [14]:
trivial_vacuum_env = TrivialVacuumEnvironment()

random_agent = Agent(program=RandomAgentProgram(['Right', 'Left', 'Suck', 'NoOp']))
print()
print("                  <<<<<<<<<<<<<Random_Agent>>>>>>>>>>>>>")
print()
print(" 초기 환경의 상태: {}.".format(trivial_vacuum_env.status))
trivial_vacuum_env.add_thing(random_agent)
print(" 초기 에이전트의 위치: {}.".format(random_agent.location), '\n')

trivial_vacuum_env.run(steps=1000)


                  <<<<<<<<<<<<<Random_Agent>>>>>>>>>>>>>

 초기 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Dirty'}.
 초기 에이전트의 위치: (0, 0). 

 에이전트의 위치: (0, 0) 
 에이전트의 행동: Suck 
 환경의 퍼포먼스: 0 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Dirty'}
----------------------------------------------
 에이전트의 위치: (0, 0) 
 에이전트의 행동: Left 
 환경의 퍼포먼스: -1 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Dirty'}
----------------------------------------------
 에이전트의 위치: (0, 0) 
 에이전트의 행동: Suck 
 환경의 퍼포먼스: -1 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Dirty'}
----------------------------------------------
 에이전트의 위치: (0, 0) 
 에이전트의 행동: Left 
 환경의 퍼포먼스: -2 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Dirty'}
----------------------------------------------
 에이전트의 위치: (0, 0) 
 에이전트의 행동: NoOp 
 환경의 퍼포먼스: -2 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Dirty'}
----------------------------------------------
 에이전트의 위치: (0, 0) 
 에이전트의 행동: NoOp 
 환경의 퍼포먼스: -2 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Dirty'}
----------------------------------------------
 에이전트의 위치: (0, 0) 

 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (0, 0) 
 에이전트의 행동: NoOp 
 환경의 퍼포먼스: -141 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (1, 0) 
 에이전트의 행동: Right 
 환경의 퍼포먼스: -142 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (1, 0) 
 에이전트의 행동: Right 
 환경의 퍼포먼스: -143 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (0, 0) 
 에이전트의 행동: Left 
 환경의 퍼포먼스: -144 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (0, 0) 
 에이전트의 행동: Left 
 환경의 퍼포먼스: -145 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (1, 0) 
 에이전트의 행동: Right 
 환경의 퍼포먼스: -146 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (1, 0) 
 에이전트의 행동: Suck 
 환경의 퍼

 에이전트의 행동: Left 
 환경의 퍼포먼스: -282 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (1, 0) 
 에이전트의 행동: Right 
 환경의 퍼포먼스: -283 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (1, 0) 
 에이전트의 행동: Suck 
 환경의 퍼포먼스: -283 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (1, 0) 
 에이전트의 행동: Right 
 환경의 퍼포먼스: -284 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (1, 0) 
 에이전트의 행동: Suck 
 환경의 퍼포먼스: -284 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (0, 0) 
 에이전트의 행동: Left 
 환경의 퍼포먼스: -285 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (1, 0) 
 에이전트의 행동: Right 
 환경의 퍼포먼스: -286 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위

 환경의 퍼포먼스: -416 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (1, 0) 
 에이전트의 행동: Right 
 환경의 퍼포먼스: -417 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (0, 0) 
 에이전트의 행동: Left 
 환경의 퍼포먼스: -418 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (0, 0) 
 에이전트의 행동: NoOp 
 환경의 퍼포먼스: -418 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (0, 0) 
 에이전트의 행동: Suck 
 환경의 퍼포먼스: -418 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (0, 0) 
 에이전트의 행동: Suck 
 환경의 퍼포먼스: -418 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (0, 0) 
 에이전트의 행동: Suck 
 환경의 퍼포먼스: -418 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (1, 0) 
 에이전트의 행

In [15]:
count_res = Counter(trivial_vacuum_env.results)
print(count_res, '\n')
for i in count_res:
    print((i, count_res[i] / len(trivial_vacuum_env.results))) 

Counter({'Suck': 255, 'NoOp': 251, 'Right': 249, 'Left': 245}) 

('Suck', 0.255)
('Left', 0.245)
('NoOp', 0.251)
('Right', 0.249)


## 3.2 TABLE-DRIVEN AGENT PROGRAM

In [16]:
table = {((loc_A, 'Clean'),): 'Right',
             ((loc_A, 'Dirty'),): 'Suck',
             ((loc_B, 'Clean'),): 'Left',
             ((loc_B, 'Dirty'),): 'Suck',
             ((loc_A, 'Dirty'), (loc_A, 'Clean')): 'Right',
             ((loc_A, 'Clean'), (loc_B, 'Dirty')): 'Suck',
             ((loc_B, 'Clean'), (loc_A, 'Dirty')): 'Suck',
             ((loc_B, 'Dirty'), (loc_B, 'Clean')): 'Left',
             ((loc_A, 'Dirty'), (loc_A, 'Clean'), (loc_B, 'Dirty')): 'Suck',
             ((loc_B, 'Dirty'), (loc_B, 'Clean'), (loc_A, 'Dirty')): 'Suck'
        }

In [17]:
trivial_vacuum_env = TrivialVacuumEnvironment()
table_driven_agent = Agent(program=TableDrivenAgentProgram(table=table))
print()
print("                  <<<<<<<<<<<<<Table_Driven_Agent>>>>>>>>>>>>>")
print()
print(" 초기 환경의 상태: {}.".format(trivial_vacuum_env.status))
trivial_vacuum_env.add_thing(table_driven_agent)
print(" 초기 에이전트 위치 {}.".format(table_driven_agent.location), '\n')

trivial_vacuum_env.run()


                  <<<<<<<<<<<<<Table_Driven_Agent>>>>>>>>>>>>>

 초기 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Dirty'}.
 초기 에이전트 위치 (0, 0). 

 에이전트의 위치: (1, 0) 
 에이전트의 행동: Right 
 환경의 퍼포먼스: -1 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Dirty'}
----------------------------------------------
 에이전트의 위치: (1, 0) 
 에이전트의 행동: Suck 
 환경의 퍼포먼스: 9 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (1, 0) 
 에이전트의 행동: None 
 환경의 퍼포먼스: 9 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (1, 0) 
 에이전트의 행동: None 
 환경의 퍼포먼스: 9 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (1, 0) 
 에이전트의 행동: None 
 환경의 퍼포먼스: 9 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (1, 0) 
 에이전트의 행동: None 
 환경의 퍼포먼스: 9 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (1, 0)

 에이전트의 위치: (1, 0) 
 에이전트의 행동: None 
 환경의 퍼포먼스: 9 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (1, 0) 
 에이전트의 행동: None 
 환경의 퍼포먼스: 9 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (1, 0) 
 에이전트의 행동: None 
 환경의 퍼포먼스: 9 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (1, 0) 
 에이전트의 행동: None 
 환경의 퍼포먼스: 9 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (1, 0) 
 에이전트의 행동: None 
 환경의 퍼포먼스: 9 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (1, 0) 
 에이전트의 행동: None 
 환경의 퍼포먼스: 9 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (1, 0) 
 에이전트의 행동: None 
 환경의 퍼포먼스: 9 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (1

 환경의 퍼포먼스: 9 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (1, 0) 
 에이전트의 행동: None 
 환경의 퍼포먼스: 9 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (1, 0) 
 에이전트의 행동: None 
 환경의 퍼포먼스: 9 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (1, 0) 
 에이전트의 행동: None 
 환경의 퍼포먼스: 9 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (1, 0) 
 에이전트의 행동: None 
 환경의 퍼포먼스: 9 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (1, 0) 
 에이전트의 행동: None 
 환경의 퍼포먼스: 9 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (1, 0) 
 에이전트의 행동: None 
 환경의 퍼포먼스: 9 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (1, 0) 
 에이전트의 행동: None 
 환경의 퍼포먼스: 9 

 환경의 퍼포먼스: 9 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (1, 0) 
 에이전트의 행동: None 
 환경의 퍼포먼스: 9 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (1, 0) 
 에이전트의 행동: None 
 환경의 퍼포먼스: 9 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (1, 0) 
 에이전트의 행동: None 
 환경의 퍼포먼스: 9 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (1, 0) 
 에이전트의 행동: None 
 환경의 퍼포먼스: 9 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (1, 0) 
 에이전트의 행동: None 
 환경의 퍼포먼스: 9 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (1, 0) 
 에이전트의 행동: None 
 환경의 퍼포먼스: 9 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (1, 0) 
 에이전트의 행동: None 
 환경의 퍼포먼스: 9 

 에이전트의 행동: None 
 환경의 퍼포먼스: 9 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (1, 0) 
 에이전트의 행동: None 
 환경의 퍼포먼스: 9 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (1, 0) 
 에이전트의 행동: None 
 환경의 퍼포먼스: 9 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (1, 0) 
 에이전트의 행동: None 
 환경의 퍼포먼스: 9 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (1, 0) 
 에이전트의 행동: None 
 환경의 퍼포먼스: 9 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (1, 0) 
 에이전트의 행동: None 
 환경의 퍼포먼스: 9 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (1, 0) 
 에이전트의 행동: None 
 환경의 퍼포먼스: 9 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (1, 0) 
 에이전트의 행동: No

 환경의 퍼포먼스: 9 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (1, 0) 
 에이전트의 행동: None 
 환경의 퍼포먼스: 9 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (1, 0) 
 에이전트의 행동: None 
 환경의 퍼포먼스: 9 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (1, 0) 
 에이전트의 행동: None 
 환경의 퍼포먼스: 9 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (1, 0) 
 에이전트의 행동: None 
 환경의 퍼포먼스: 9 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (1, 0) 
 에이전트의 행동: None 
 환경의 퍼포먼스: 9 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (1, 0) 
 에이전트의 행동: None 
 환경의 퍼포먼스: 9 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (1, 0) 
 에이전트의 행동: None 
 환경의 퍼포먼스: 9 

 환경의 퍼포먼스: 9 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (1, 0) 
 에이전트의 행동: None 
 환경의 퍼포먼스: 9 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (1, 0) 
 에이전트의 행동: None 
 환경의 퍼포먼스: 9 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (1, 0) 
 에이전트의 행동: None 
 환경의 퍼포먼스: 9 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (1, 0) 
 에이전트의 행동: None 
 환경의 퍼포먼스: 9 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (1, 0) 
 에이전트의 행동: None 
 환경의 퍼포먼스: 9 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (1, 0) 
 에이전트의 행동: None 
 환경의 퍼포먼스: 9 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (1, 0) 
 에이전트의 행동: None 
 환경의 퍼포먼스: 9 

 에이전트의 행동: None 
 환경의 퍼포먼스: 9 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (1, 0) 
 에이전트의 행동: None 
 환경의 퍼포먼스: 9 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (1, 0) 
 에이전트의 행동: None 
 환경의 퍼포먼스: 9 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (1, 0) 
 에이전트의 행동: None 
 환경의 퍼포먼스: 9 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (1, 0) 
 에이전트의 행동: None 
 환경의 퍼포먼스: 9 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (1, 0) 
 에이전트의 행동: None 
 환경의 퍼포먼스: 9 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (1, 0) 
 에이전트의 행동: None 
 환경의 퍼포먼스: 9 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (1, 0) 
 에이전트의 행동: No

In [18]:
count_res = Counter(trivial_vacuum_env.results)
print(count_res, '\n')
for i in count_res:
    print((i, count_res[i] / len(trivial_vacuum_env.results))) 

Counter({None: 998, 'Right': 1, 'Suck': 1}) 

('Right', 0.001)
('Suck', 0.001)
(None, 0.998)


## 3.3 SIMPLE REFLEX AGENT PROGRAM

In [19]:
trivial_vacuum_env = TrivialVacuumEnvironment()


def SimpleReflexAgentProgram():  
    def program(percept):
        loc, status = percept
        return ('Suck' if status == 'Dirty' else'Right' if loc == loc_A else'Left')
    return program
        
program = SimpleReflexAgentProgram()
simple_reflex_agent = Agent(program)

print()
print("                  <<<<<<<<<<<<<Simple_Reflex_Agent>>>>>>>>>>>>>")
print()
trivial_vacuum_env.add_thing(simple_reflex_agent)
print(" 초기 에이전트 위치: {}.".format(simple_reflex_agent.location))
print(" 초기 환경의 상태: {}.".format(trivial_vacuum_env.status), '\n')

trivial_vacuum_env.run()


                  <<<<<<<<<<<<<Simple_Reflex_Agent>>>>>>>>>>>>>

 초기 에이전트 위치: (1, 0).
 초기 환경의 상태: {(0, 0): 'Dirty', (1, 0): 'Clean'}. 

 에이전트의 위치: (0, 0) 
 에이전트의 행동: Left 
 환경의 퍼포먼스: -1 
 환경의 상태: {(0, 0): 'Dirty', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (0, 0) 
 에이전트의 행동: Suck 
 환경의 퍼포먼스: 9 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (1, 0) 
 에이전트의 행동: Right 
 환경의 퍼포먼스: 8 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (0, 0) 
 에이전트의 행동: Left 
 환경의 퍼포먼스: 7 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (1, 0) 
 에이전트의 행동: Right 
 환경의 퍼포먼스: 6 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (0, 0) 
 에이전트의 행동: Left 
 환경의 퍼포먼스: 5 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (1,

 환경의 퍼포먼스: -98 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (0, 0) 
 에이전트의 행동: Left 
 환경의 퍼포먼스: -99 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (1, 0) 
 에이전트의 행동: Right 
 환경의 퍼포먼스: -100 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (0, 0) 
 에이전트의 행동: Left 
 환경의 퍼포먼스: -101 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (1, 0) 
 에이전트의 행동: Right 
 환경의 퍼포먼스: -102 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (0, 0) 
 에이전트의 행동: Left 
 환경의 퍼포먼스: -103 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (1, 0) 
 에이전트의 행동: Right 
 환경의 퍼포먼스: -104 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (0, 0) 
 에이전트의 행

----------------------------------------------
 에이전트의 위치: (1, 0) 
 에이전트의 행동: Right 
 환경의 퍼포먼스: -192 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (0, 0) 
 에이전트의 행동: Left 
 환경의 퍼포먼스: -193 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (1, 0) 
 에이전트의 행동: Right 
 환경의 퍼포먼스: -194 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (0, 0) 
 에이전트의 행동: Left 
 환경의 퍼포먼스: -195 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (1, 0) 
 에이전트의 행동: Right 
 환경의 퍼포먼스: -196 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (0, 0) 
 에이전트의 행동: Left 
 환경의 퍼포먼스: -197 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (1, 0) 
 에이전트의 행동: Right 
 환경의 퍼포먼스: -198 
 환경의 상태: {(0, 0): 'Clean', (1, 0

 에이전트의 행동: Right 
 환경의 퍼포먼스: -272 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (0, 0) 
 에이전트의 행동: Left 
 환경의 퍼포먼스: -273 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (1, 0) 
 에이전트의 행동: Right 
 환경의 퍼포먼스: -274 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (0, 0) 
 에이전트의 행동: Left 
 환경의 퍼포먼스: -275 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (1, 0) 
 에이전트의 행동: Right 
 환경의 퍼포먼스: -276 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (0, 0) 
 에이전트의 행동: Left 
 환경의 퍼포먼스: -277 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (1, 0) 
 에이전트의 행동: Right 
 환경의 퍼포먼스: -278 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 

 환경의 퍼포먼스: -386 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (0, 0) 
 에이전트의 행동: Left 
 환경의 퍼포먼스: -387 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (1, 0) 
 에이전트의 행동: Right 
 환경의 퍼포먼스: -388 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (0, 0) 
 에이전트의 행동: Left 
 환경의 퍼포먼스: -389 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (1, 0) 
 에이전트의 행동: Right 
 환경의 퍼포먼스: -390 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (0, 0) 
 에이전트의 행동: Left 
 환경의 퍼포먼스: -391 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (1, 0) 
 에이전트의 행동: Right 
 환경의 퍼포먼스: -392 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (0, 0) 
 에이전트의

 에이전트의 위치: (1, 0) 
 에이전트의 행동: Right 
 환경의 퍼포먼스: -460 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (0, 0) 
 에이전트의 행동: Left 
 환경의 퍼포먼스: -461 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (1, 0) 
 에이전트의 행동: Right 
 환경의 퍼포먼스: -462 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (0, 0) 
 에이전트의 행동: Left 
 환경의 퍼포먼스: -463 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (1, 0) 
 에이전트의 행동: Right 
 환경의 퍼포먼스: -464 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (0, 0) 
 에이전트의 행동: Left 
 환경의 퍼포먼스: -465 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (1, 0) 
 에이전트의 행동: Right 
 환경의 퍼포먼스: -466 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
-----------------------------------

 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (1, 0) 
 에이전트의 행동: Right 
 환경의 퍼포먼스: -546 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (0, 0) 
 에이전트의 행동: Left 
 환경의 퍼포먼스: -547 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (1, 0) 
 에이전트의 행동: Right 
 환경의 퍼포먼스: -548 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (0, 0) 
 에이전트의 행동: Left 
 환경의 퍼포먼스: -549 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (1, 0) 
 에이전트의 행동: Right 
 환경의 퍼포먼스: -550 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (0, 0) 
 에이전트의 행동: Left 
 환경의 퍼포먼스: -551 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (1, 0) 
 에이전트의 행동: Right 
 환경의 

 환경의 퍼포먼스: -704 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (0, 0) 
 에이전트의 행동: Left 
 환경의 퍼포먼스: -705 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (1, 0) 
 에이전트의 행동: Right 
 환경의 퍼포먼스: -706 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (0, 0) 
 에이전트의 행동: Left 
 환경의 퍼포먼스: -707 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (1, 0) 
 에이전트의 행동: Right 
 환경의 퍼포먼스: -708 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (0, 0) 
 에이전트의 행동: Left 
 환경의 퍼포먼스: -709 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (1, 0) 
 에이전트의 행동: Right 
 환경의 퍼포먼스: -710 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (0, 0) 
 에이전트의

----------------------------------------------
 에이전트의 위치: (0, 0) 
 에이전트의 행동: Left 
 환경의 퍼포먼스: -845 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (1, 0) 
 에이전트의 행동: Right 
 환경의 퍼포먼스: -846 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (0, 0) 
 에이전트의 행동: Left 
 환경의 퍼포먼스: -847 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (1, 0) 
 에이전트의 행동: Right 
 환경의 퍼포먼스: -848 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (0, 0) 
 에이전트의 행동: Left 
 환경의 퍼포먼스: -849 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (1, 0) 
 에이전트의 행동: Right 
 환경의 퍼포먼스: -850 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (0, 0) 
 에이전트의 행동: Left 
 환경의 퍼포먼스: -851 
 환경의 상태: {(0, 0): 'Clean', (1, 0)

 환경의 퍼포먼스: -974 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (0, 0) 
 에이전트의 행동: Left 
 환경의 퍼포먼스: -975 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (1, 0) 
 에이전트의 행동: Right 
 환경의 퍼포먼스: -976 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (0, 0) 
 에이전트의 행동: Left 
 환경의 퍼포먼스: -977 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (1, 0) 
 에이전트의 행동: Right 
 환경의 퍼포먼스: -978 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (0, 0) 
 에이전트의 행동: Left 
 환경의 퍼포먼스: -979 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (1, 0) 
 에이전트의 행동: Right 
 환경의 퍼포먼스: -980 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (0, 0) 
 에이전트의

In [20]:
count_res = Counter(trivial_vacuum_env.results)
print(count_res, '\n')
for i in count_res:
    print((i, count_res[i] / len(trivial_vacuum_env.results))) 

Counter({'Left': 500, 'Right': 499, 'Suck': 1}) 

('Left', 0.5)
('Suck', 0.001)
('Right', 0.499)


## 3.4 MODEL-BASED REFLEX AGENT PROGRAM

In [21]:
trivial_vacuum_env = TrivialVacuumEnvironment()

def ModelBasedVacuumAgent():
    model = {loc_A: None, loc_B: None}
    def program(percept):
        location, status = percept
        model[location] = status  # Update the model here
        if model[loc_A] == model[loc_B] == 'Clean':
            return 'NoOp'
        elif status == 'Dirty':
            return 'Suck'
        elif location == loc_A:
            return 'Right'
        elif location == loc_B:
            return 'Left'

    return Agent(program)

print()
print("                  <<<<<<<<<<<<<Model_Based_Reflex_Agent>>>>>>>>>>>>>")
print()
model_based_reflex_agent = ModelBasedVacuumAgent()

trivial_vacuum_env.add_thing(model_based_reflex_agent)
print(" 초기 에이전트의 위치 {}.".format(model_based_reflex_agent.location))
print(" 초기 환경의 상태: {}.".format(trivial_vacuum_env.status), '\n')


trivial_vacuum_env.run()


                  <<<<<<<<<<<<<Model_Based_Reflex_Agent>>>>>>>>>>>>>

 초기 에이전트의 위치 (1, 0).
 초기 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Dirty'}. 

 에이전트의 위치: (1, 0) 
 에이전트의 행동: Suck 
 환경의 퍼포먼스: 10 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (0, 0) 
 에이전트의 행동: Left 
 환경의 퍼포먼스: 9 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (0, 0) 
 에이전트의 행동: NoOp 
 환경의 퍼포먼스: 9 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (0, 0) 
 에이전트의 행동: NoOp 
 환경의 퍼포먼스: 9 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (0, 0) 
 에이전트의 행동: NoOp 
 환경의 퍼포먼스: 9 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (0, 0) 
 에이전트의 행동: NoOp 
 환경의 퍼포먼스: 9 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: 

 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (0, 0) 
 에이전트의 행동: NoOp 
 환경의 퍼포먼스: 9 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (0, 0) 
 에이전트의 행동: NoOp 
 환경의 퍼포먼스: 9 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (0, 0) 
 에이전트의 행동: NoOp 
 환경의 퍼포먼스: 9 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (0, 0) 
 에이전트의 행동: NoOp 
 환경의 퍼포먼스: 9 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (0, 0) 
 에이전트의 행동: NoOp 
 환경의 퍼포먼스: 9 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (0, 0) 
 에이전트의 행동: NoOp 
 환경의 퍼포먼스: 9 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (0, 0) 
 에이전트의 행동: NoOp 
 환경의 퍼포먼스: 9 
 환경의 상태: {(0,

 환경의 퍼포먼스: 9 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (0, 0) 
 에이전트의 행동: NoOp 
 환경의 퍼포먼스: 9 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (0, 0) 
 에이전트의 행동: NoOp 
 환경의 퍼포먼스: 9 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (0, 0) 
 에이전트의 행동: NoOp 
 환경의 퍼포먼스: 9 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (0, 0) 
 에이전트의 행동: NoOp 
 환경의 퍼포먼스: 9 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (0, 0) 
 에이전트의 행동: NoOp 
 환경의 퍼포먼스: 9 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (0, 0) 
 에이전트의 행동: NoOp 
 환경의 퍼포먼스: 9 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (0, 0) 
 에이전트의 행동: NoOp 
 환경의 퍼포먼스: 9 

 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (0, 0) 
 에이전트의 행동: NoOp 
 환경의 퍼포먼스: 9 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (0, 0) 
 에이전트의 행동: NoOp 
 환경의 퍼포먼스: 9 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (0, 0) 
 에이전트의 행동: NoOp 
 환경의 퍼포먼스: 9 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (0, 0) 
 에이전트의 행동: NoOp 
 환경의 퍼포먼스: 9 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (0, 0) 
 에이전트의 행동: NoOp 
 환경의 퍼포먼스: 9 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (0, 0) 
 에이전트의 행동: NoOp 
 환경의 퍼포먼스: 9 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (0, 0) 
 에이전트의 행동: NoOp 
 환경의 퍼포먼스: 9 
 환경의 상태: {(0,

----------------------------------------------
 에이전트의 위치: (0, 0) 
 에이전트의 행동: NoOp 
 환경의 퍼포먼스: 9 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (0, 0) 
 에이전트의 행동: NoOp 
 환경의 퍼포먼스: 9 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (0, 0) 
 에이전트의 행동: NoOp 
 환경의 퍼포먼스: 9 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (0, 0) 
 에이전트의 행동: NoOp 
 환경의 퍼포먼스: 9 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (0, 0) 
 에이전트의 행동: NoOp 
 환경의 퍼포먼스: 9 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (0, 0) 
 에이전트의 행동: NoOp 
 환경의 퍼포먼스: 9 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (0, 0) 
 에이전트의 행동: NoOp 
 환경의 퍼포먼스: 9 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
-------------

 환경의 퍼포먼스: 9 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (0, 0) 
 에이전트의 행동: NoOp 
 환경의 퍼포먼스: 9 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (0, 0) 
 에이전트의 행동: NoOp 
 환경의 퍼포먼스: 9 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (0, 0) 
 에이전트의 행동: NoOp 
 환경의 퍼포먼스: 9 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (0, 0) 
 에이전트의 행동: NoOp 
 환경의 퍼포먼스: 9 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (0, 0) 
 에이전트의 행동: NoOp 
 환경의 퍼포먼스: 9 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (0, 0) 
 에이전트의 행동: NoOp 
 환경의 퍼포먼스: 9 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (0, 0) 
 에이전트의 행동: NoOp 
 환경의 퍼포먼스: 9 

----------------------------------------------
 에이전트의 위치: (0, 0) 
 에이전트의 행동: NoOp 
 환경의 퍼포먼스: 9 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (0, 0) 
 에이전트의 행동: NoOp 
 환경의 퍼포먼스: 9 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (0, 0) 
 에이전트의 행동: NoOp 
 환경의 퍼포먼스: 9 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (0, 0) 
 에이전트의 행동: NoOp 
 환경의 퍼포먼스: 9 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (0, 0) 
 에이전트의 행동: NoOp 
 환경의 퍼포먼스: 9 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (0, 0) 
 에이전트의 행동: NoOp 
 환경의 퍼포먼스: 9 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (0, 0) 
 에이전트의 행동: NoOp 
 환경의 퍼포먼스: 9 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
-------------

 환경의 퍼포먼스: 9 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (0, 0) 
 에이전트의 행동: NoOp 
 환경의 퍼포먼스: 9 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (0, 0) 
 에이전트의 행동: NoOp 
 환경의 퍼포먼스: 9 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (0, 0) 
 에이전트의 행동: NoOp 
 환경의 퍼포먼스: 9 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (0, 0) 
 에이전트의 행동: NoOp 
 환경의 퍼포먼스: 9 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (0, 0) 
 에이전트의 행동: NoOp 
 환경의 퍼포먼스: 9 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (0, 0) 
 에이전트의 행동: NoOp 
 환경의 퍼포먼스: 9 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (0, 0) 
 에이전트의 행동: NoOp 
 환경의 퍼포먼스: 9 

 에이전트의 행동: NoOp 
 환경의 퍼포먼스: 9 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (0, 0) 
 에이전트의 행동: NoOp 
 환경의 퍼포먼스: 9 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (0, 0) 
 에이전트의 행동: NoOp 
 환경의 퍼포먼스: 9 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (0, 0) 
 에이전트의 행동: NoOp 
 환경의 퍼포먼스: 9 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (0, 0) 
 에이전트의 행동: NoOp 
 환경의 퍼포먼스: 9 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (0, 0) 
 에이전트의 행동: NoOp 
 환경의 퍼포먼스: 9 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (0, 0) 
 에이전트의 행동: NoOp 
 환경의 퍼포먼스: 9 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (0, 0) 
 에이전트의 행동: No

 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (0, 0) 
 에이전트의 행동: NoOp 
 환경의 퍼포먼스: 9 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (0, 0) 
 에이전트의 행동: NoOp 
 환경의 퍼포먼스: 9 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (0, 0) 
 에이전트의 행동: NoOp 
 환경의 퍼포먼스: 9 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (0, 0) 
 에이전트의 행동: NoOp 
 환경의 퍼포먼스: 9 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (0, 0) 
 에이전트의 행동: NoOp 
 환경의 퍼포먼스: 9 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (0, 0) 
 에이전트의 행동: NoOp 
 환경의 퍼포먼스: 9 
 환경의 상태: {(0, 0): 'Clean', (1, 0): 'Clean'}
----------------------------------------------
 에이전트의 위치: (0, 0) 
 에이전트의 행동: NoOp 
 환경의 퍼포먼스: 9 
 환경의 상태: {(0,

In [22]:
count_res = Counter(trivial_vacuum_env.results)
print(count_res, '\n')
for i in count_res:
    print((i, count_res[i] / len(trivial_vacuum_env.results))) 

Counter({'NoOp': 998, 'Suck': 1, 'Left': 1}) 

('Suck', 0.001)
('Left', 0.001)
('NoOp', 0.998)
